# Lab 2: Machine Learning
#### Team Members: Alfonso De Vera, Joel Karakkattu, Loretta Malinovic, Alexandra Monte

All imports for tasks

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import os
import pathlib


#### Task 01 – Generate Class Names

Use NumPy’s `np.array()` to create a list of class names from the subdirectories in your training folder.


In [5]:
data_dir = pathlib.Path("datagiven/data/CIFAR-10-images/train/")
class_names = np.array(sorted([p.name for p in data_dir.iterdir() if p.is_dir()]))
print(class_names, len(class_names))


['airplane' 'automobile' 'bird' 'cat' 'deer' 'dog' 'frog' 'horse' 'ship'
 'truck'] 10


Now associate each class name with a numeric label for use in machine learning.


In [6]:
class_names_dict = {name: idx for idx, name in enumerate(class_names)}
len(class_names_dict), class_names_dict


(10,
 {np.str_('airplane'): 0,
  np.str_('automobile'): 1,
  np.str_('bird'): 2,
  np.str_('cat'): 3,
  np.str_('deer'): 4,
  np.str_('dog'): 5,
  np.str_('frog'): 6,
  np.str_('horse'): 7,
  np.str_('ship'): 8,
  np.str_('truck'): 9})